# HW3 - Q3 [35 pts]

## Important Notices

`user()`

`bucket()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

</div>

`load_data()`

`main()`

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1616189033226_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [3]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

from pyspark.sql.functions import col
from pyspark.sql import *

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-spring2021"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp,lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function
</div>

## 3a. [1 pt] Update the `user()` function

In [4]:
def user():
    return 'mganesan7'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [23]:
def long_trips(trips):
    # Returns a Dataframe with Schema the same as :trips:
    
    long_trip = trips[trips['trip_distance'] >= 2]
    return long_trip

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [42]:
def manhattan_trips(trips, lookup):
    # Returns a Dataframe with Schema: DOLocationID, pcount
    
    # Considering only the trips that are greater than 2 miles
    trips_filtered = long_trips(trips)
    
    #Code to extact top 20 DO Locations based on passenger count for Manhattan borough
    manhattan_trips = trips_filtered.join(lookup, trips_filtered.DOLocationID == lookup.LocationID,how='inner') \
                           .filter(col('Borough') == 'Manhattan') \
                           .groupBy(col('DOLocationID')).agg({'passenger_count':'sum'}) \
                           .withColumnRenamed('sum(passenger_count)','pcount') \
                           .orderBy(col('pcount').desc()) \
                           .limit(20)
    return manhattan_trips

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [139]:
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    
    
    trips_filtered = long_trips(trips)

    # Calculating the average total amount per trip for each Pickup Location

    avg_tot_amount_per_trip = trips_filtered.groupBy(col('PULocationID')).agg({'total_amount':'avg'}) \
                                        .withColumnRenamed('avg(total_amount)','avg_total_amount')

    #Calculating the total count of trips for each pickup location
    total_count = trips_filtered.groupBy(col('PULocationID')).count() \
                            .withColumnRenamed('count','total_count')

    # Getting the top 20 drop off locations as a list for further processing
    do_top20_locations = mtrips.select('DOLocationID').collect()

    # Identifying all trips that end in one of the 20 drop-off locations
    count_popular = trips_filtered.where(col('DOLocationID').isin([value['DOLocationID'] for value in do_top20_locations])) \
                              .groupBy(col('PULocationID')).count() \
                              .withColumnRenamed('count','popular_count')

    #Combining the dataframes for further processing
    df_final = avg_tot_amount_per_trip.join(total_count,["PULocationID"],how="inner") \
                                      .join(count_popular, ["PULocationID"], how="left")

    df_return = df_final.withColumn('weighted_profit', ((col('popular_count')/col('total_count'))*col('avg_total_amount')).cast("decimal(17,14)")    ) \
                        .select("PULocationID","weighted_profit")
    return df_return

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [150]:
def final_output(calc, lookup): 
    # Returns a Dataframe with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe

    final_df = calc.join(lookup, calc.PULocationID == lookup.LocationID, how='inner') \
        .orderBy(col('weighted_profit').desc()) \
        .select('Zone','Borough','weighted_profit') \
        .limit(20)
    
    return final_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-info">
    Test your code on the small dataset first, as the large dataset will take a significantly longer time to run
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> use the same bucket url for multiple runs of the `main()` function, as this will cause errors. Make sure to change the name of your output location every time. (ie: s3://cse6242-gburdell3/output-small2)
</div>

Update the below cell with the path to your bucket, then run the below cell to run your code to store the results in S3.

When you have confirmed the results of the small dataset, run it again using the large dataset. Your output file will appear in a folder in your s3 bucket called YOUROUTPUT.csv as a csv file with a name something like part-0000-4d992f7a-0ad3-48f8-8c72-0022984e4b50-c000.csv. Download this file and rename it to q3_output.csv for submission. Do not make any other changes to the file. 

In [154]:
# update your bucket path
#bucket = 's3://cse6242-mganesan7/output-small'
#main('small', bucket)

bucket = 's3://cse6242-mganesan7/output-large'
main('large', bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

mganesan7
Trip Count:  187203269
+--------------------+-------------+-----------------+
|                Zone|      Borough|  weighted_profit|
+--------------------+-------------+-----------------+
|        Baisley Park|       Queens|29.36045577913085|
|Flushing Meadows-...|       Queens|27.30484573361768|
|       South Jamaica|       Queens|26.29491623987349|
|     Randalls Island|    Manhattan|24.15098994022752|
|        Astoria Park|       Queens|21.70641711214752|
|Briarwood/Jamaica...|       Queens|19.94506463178933|
|Springfield Garde...|       Queens|19.46830928878191|
|             Jamaica|       Queens|19.28394300013789|
|              Corona|       Queens|18.22876924815597|
|   LaGuardia Airport|       Queens|18.18133880837299|
|         Jamaica Bay|       Queens|17.10052944675790|
|             Maspeth|       Queens|17.00545064007955|
|Eltingville/Annad...|Staten Island|16.83776475694445|
|         JFK Airport|       Queens|16.77772534824964|
|        Battery Park|    Manhat

#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading
</div>